### Imports

In [1]:
import json   
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
 import json
 !gdown --id 1xWkZ70Fc0y0WvzI-4Iw8fbPI9hP_biZT
 with open('/content/tops_fashion.json') as f:
      data = json.load(f)
   

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1xWkZ70Fc0y0WvzI-4Iw8fbPI9hP_biZT 



FileNotFoundError: ignored

## Exploring the data

In [ ]:
df = pd.DataFrame(data)
df = df.drop(['sku', 'author', 'publisher','availability', 'reviews', 'large_image_url','availability_type','small_image_url','editorial_review','model','manufacturer'], axis=1)
df.head()
del data
#asin - amazon standard identification number

In [ ]:
for i in range(10):
  print(df['title'][i]);
  plt.figure

In [ ]:
print('Attributes in data')
for col in df.columns:
  print('->',col)
print('\nDuplicate in data, and nan values')  
for col in df.columns:
  print('-> Column: {}, Total values {}, unique values {}, nan values {}'.format(col, len(df[col]), len(set(list(df[col]))),sum(df[col].isnull().values)))  

## Recommendation on basis of titles of clothes
1. Remove all products with very few words in there title.
2. Sort the whole data based on title (alphabetical order of title),m and then remove titles that are very similar
3. using TFIDF(Term Frequency inverse document frequency) on Product titles to get an array representation as it gives less weightage to the words that appear often in the documents and focuses on words that are descriptive of the image

In [ ]:
print('Total elements in the list: ', len(df))
print('Attributes of an apparel: ', list(df.columns))
print('Total null elements in formatter_price', sum(df['formatted_price'].isnull().values))

In [ ]:
df = df.sort_values('title')
df.reset_index(drop = True, inplace = True)

In [ ]:
df.head(2) #Used this command to showcase the first two rows/entries in the database. 

### **Cleaning the texts**

### Remove duplicates in data

In [ ]:
df = df.drop_duplicates(subset = 'medium_image_url').reset_index().drop(['index'],axis=1)
df = df.drop_duplicates(subset = 'title').reset_index().drop(['index'],axis=1)
print(len(df))

### Remove null entries

In [ ]:
df =df.loc[~df['color'].isnull()]
df =df.loc[~df['title'].isnull()]
df =df.loc[~df['formatted_price'].isnull()]
len(df)

### Remove stop words

In [ ]:
stopwrds = list(stopwords.words('english'))
l = list(df['title'])
for i in range(len(df['title'])):
  for j in range(len(stopwrds)):
    l[i] = l[i].replace(' '+stopwrds[j]+' ',' ')
df['title']=l

In [ ]:
tokenizer = Tokenizer(oov_token="<UNK>") # if num words not provided it consider all
tokenizer.fit_on_texts(list(df['title']))      #this is a must, to give the tokenizer an idea of the train data
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(list(df['title']))
sequences[0]
tok_sent = tokenizer.sequences_to_texts(sequences)
df['title']=tok_sent

In [ ]:
for col in df.columns:
  print('Column: {}, Total values {}, unique values {}, nan values {}'.format(col, len(df[col]), len(set(list(df[col]))),sum(df[col].isnull().values)))

In [ ]:
df.reset_index(drop=True,inplace=True)
indices = list(df.index)
df.head(1)

* Removing titles that have are very similar

In [ ]:
import itertools
deduped_idx = []
i = 0
j = 0 
while i < len(df) and j < len(df):
    previous_i = i
    a = df['title'].loc[i].split()
    j = i+1
    while j < len(df):
        b = df['title'].loc[indices[j]].split()
        length = max(len(a), len(b))
        count  = 0
        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1
        if (length - count) > 2: 
            deduped_idx.append(i)
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break

In [ ]:
df = df.iloc[deduped_idx]
df.reset_index(drop=True,inplace=True)
df.head(2)
list_titles = list(df['title'])
print(len(df))

In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = CountVectorizer()
vectorizer
X = vectorizer.fit_transform(list_titles) #corpus is a list tof sentences
analyze = vectorizer.build_analyzer()

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(list_titles)

In [ ]:
def closestkrecom(idx,k):
  dist = sklearn.metrics.pairwise_distances( X, X[idx], metric='cosine')
  dist = np.squeeze(dist)
  #print(dist.shape)
  sort_idxs = np.random.choice(np.argsort(dist,axis=0)[1:],k)
  return sort_idxs

In [ ]:
import random
import tensorflow as tf

In [ ]:
for idx in random.sample(list(np.arange(len(df))), 20):
  print('-->  ', df['title'][idx])
  for j in closestkrecom(idx,5):
    print('   ',df['title'][j])
  print()

## Recommendation on basis of image
* I am going to use an autoencoder and get a 2000 dimensional embedding
* Then 



![link text](https://miro.medium.com/max/3148/1*44eDEuZBEsmG_TCAKRI3Kw@2x.png)


### Imports and downloading the images

In [3]:
import os
from PIL import Image
import requests
from io import BytesIO
import skimage
from skimage import io
import urllib.request
import cv2
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
path =  "/content/drive/My Drive/Apparel Reco"
try:
  os.makedirs('/content/Dataset/Train/Sample')                 #better than mkdir
except OSError:
  pass
try:
  os.makedirs('/content/Dataset/Test/Sample')                 #better than mkdir
except OSError:
  pass  

In [ ]:
def download_images(df): #function to download all the images
  j=0
  o=0
  train,test = train_test_split(np.arange(len(df)), test_size=0.1, )
  for i in train:
    try:
      name = df['asin'][i] + ".jpg"
      print(name,', ',o)
      o+=1
      urllib.request.urlretrieve(df['medium_image_url'][i], "/content/Dataset/Train/Sample/"+ name)
    except:
      print("N/A",j)
      j+=1  
  for i in test:
    try:
      name = df['asin'][i] + ".jpg"
      print(name,', ',o)
      o+=1
      urllib.request.urlretrieve(df['medium_image_url'][i], "/content/Dataset/Test/Sample/"+ name)
    except:
      print("N/A",j)
      j+=1      
download_images(df)

In [ ]:
def disp_img(idx): #this function displays image from idx in df
  response = requests.get(df['medium_image_url'][idx])
  img = Image.open(BytesIO(response.content))
  return img

In [ ]:
def ur2arr(url, disp=False): #This takes in an url and convert it to array
  image_filename = url
  image_numpy = skimage.io.imread(image_filename)
  if disp:
    plt.imshow(image_numpy)
  return image_numpy

### Creating the Data Generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen  = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(                        #here we fit the data to the generator
        '/content/Dataset/Train',  # This is the source directory for training images
        target_size=(128, 128),  # All images will be resized to 150x150, no need to provide the channel dimension
        batch_size=64,           #labels are automatically assigned on basis of the directory they are in
        class_mode='input')
test_generator = test_datagen.flow_from_directory(                          #here we fit the data to the generator
        '/content/Dataset/Test',  # This is the source directory for test images
        target_size=(128, 128),  # All images will be resized to 150x150, no need to provide the channel dimension
        batch_size=128,           #labels are automatically assigned on basis of the directory they are in
        class_mode='input')

### Building the model

In [ ]:
input_img = tf.keras.layers.Input(shape=(128, 128, 3))  # adapt this if using `channels_first` image data format
x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
encoder = tf.keras.Model(input_img, tf.keras.layers.Flatten()(encoded))
# at this point the representation is (4, 4, 8) i.e. 128-dimensional
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(16, (1, 1), activation='relu')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
decoded = tf.keras.layers.UpSampling2D((1, 1))(x)
autoencoder = tf.keras.Model(input_img, decoded)
autoencoder.summary()
opt = tf.keras.optimizers.Adam()
autoencoder.compile(optimizer=opt, loss='binary_crossentropy')

### Training the model

In [ ]:
opt = tf.keras.optimizers.Adam(0.01)
autoencoder.compile(optimizer=opt, loss='binary_crossentropy')
history = autoencoder.fit(train_generator,
                epochs=10,
                validation_data=test_generator
                #callbacks=[tf.keras.callbacks.LearningRateScheduler(lrs)]
                )

### Image Reconstruction quality check

In [ ]:
x_test = next(iter(test_generator))
x_test[1].shape

In [ ]:
x_test = next(iter(test_generator))[0]
decoded_imgs = autoencoder.predict(x_test)
n = 10
plt.figure(figsize=(20, 4))
for i in range(n+1):
    if i==0:
      continue
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[i].reshape(128, 128,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(128, 128,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
os.listdir('/content/Dataset/Train/Sample')
p = {}
t=0
for filename in os.listdir('/content/Dataset/Train/Sample'):
  img = cv2.imread(os.path.join('/content/Dataset/Train/Sample',filename))
  img = cv2.resize(img, (128,128))
  img = np.expand_dims(img,0)
  p[filename] = encoder.predict(img)
  t+=1
  print(t)

In [ ]:
t=0
for filename in os.listdir('/content/Dataset/Test/Sample'):
  img = cv2.imread(os.path.join('/content/Dataset/Test/Sample',filename))
  img = cv2.resize(img, (128,128))
  img = np.expand_dims(img,0)
  p[filename] = encoder(img)
  t+=1
  print(t)

In [ ]:
from PIL import Image as PImage
from matplotlib import rcParams

def disp(l):
  k= len(l)
  rcParams['figure.figsize'] = 11.7,8.27
  for i,filename in enumerate(l): 
    plt.figure(figsize =(3,3))
    try:
      img = PImage.open('/content/Dataset/Test/Sample/' + filename)
      plt.imshow(img)
    except:
      if(i==0):
       plt.title('Chosen Apparel')
      # print(df['title'][i])
      else:
        plt.title('Recommended Apparels') 
        #plt.title((df['title'][i]))
      img = PImage.open('/content/Dataset/Train/Sample/' + filename)
      plt.imshow(img)

In [ ]:
from scipy.spatial import distance
def closestkrecom(idx,k):
  asin = df.iloc[idx]['asin'] + '.jpg'
  q={}
  arr = np.squeeze(p[asin])
  for key, value in p.items():
    dist = distance.cosine(arr,np.squeeze(value))
    q[key] = dist
  q = {value:key for key,value in q.items() } 
  l=[]
  for j,i in enumerate(sorted(q)): 
    if j== k:
      break  
    l.append(str(q[i])) 
  return l 

In [ ]:
temp

In [ ]:
temp = random.randint(0,len(df))
l = closestkrecom(temp,10)
disp(l)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token="<UNK>") 
tokenizer.fit_on_texts(list_titles)
word_index = tokenizer.word_index
vocab_len = len(word_index)
def sentence2indices(sentences):
  sequences = tokenizer.texts_to_sequences(sentences)
  padded = pad_sequences(sequences, maxlen=7, padding = 'post', truncating = 'post')
  return padded
import tensorflow as tf
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_len, 32, input_length=7),
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(32)])
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(sentence2indices(list_titles),)